# Definition

The value of a European call option is:

$$
C(S_t,t) = N(d_1)S_t - N(d_2)PV(K) \\
d_1 = \frac{1}{\sigma\sqrt{T-t}} \\
d_2 = d_1-\sigma\sqrt{T-t} \\
PV(K) = Ke^{r(T-t)}
$$

Under the NA0 (No Arbitrage Opportunity) the assumption the **put-call parity** exists as:

$$
C(S_t,t) - P(S_t,t) = S_t - PV(K) \\
PV(K) = Ke^{r(T-t)}
$$

The value of a European put option is:

$$
P(S_t,t) =  N(-d_2)PV(K) - N(-d_1)S_t
$$

$N(.)$ is the cumulative distribution function of the standard normal distribution <br>
$K$ is the strike price <br>
$T-t$ is the time to maturity (expressed in years)<br>
$S_t$ is the spot price of the underlying asset<br>
$r$ is the risk-free rate (annual rate, expressed in terms of continuous compounding)<br>
$\sigma$ is the implied volatility of returns of the underlying asset.<br>


In [1]:
from typing import Literal
import numpy as np
import scipy.stats as stats

from modules.pricings import (
    monte_carlo_option_price,
    from_call_to_put,
    from_put_to_call,
)


In [2]:
def black_scholes_option_price(
    s0: float | int,
    k: float | int,
    r: float | int,
    tau: float | int,
    sigma: float | int,
    option_type: Literal["call", "put"] = "call",
) -> float:
    """Monte Carlo simulation of the option price.

    Args:
        s0 (float | int): The spot price.
        k (float | int): The strike price.
        r (float | int): The risk-free interest rate.
        tau (float | int): The time to maturity : T-t.
        sigma (float | int): The volatility.
        option_type: Literal["call", "put"]: The type of option to price. Defaults to "call".

    Returns:
        float: The simulated option price.
    """
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma**2) * tau) / (sigma * tau**0.5)
    d2 = d1 - sigma * tau**0.5
    pv = k * np.exp(-r * tau)

    match option_type:
        case "put":
            return pv * stats.norm.cdf(-d2) - s0 * stats.norm.cdf(-d1)
        case "call":
            return s0 * stats.norm.cdf(d1) - pv * stats.norm.cdf(d2)
        case _:
            raise ValueError(f"Unknown option type: {option_type}")

In [3]:
call_price = black_scholes_option_price(100, 110, 0.05, 1, 0.2, "call")

print(f"Call price: $ {call_price:.2f}")

put_price = black_scholes_option_price(100, 110, 0.05, 1, 0.2, "put")
print(f"Put price using the BS formula: $ {put_price:.2f}")
print(
    f"Put price using the put call parity formula: $ {from_call_to_put(call_price,100,110,0.05,1):.2f}"
)


Call price: $ 6.04
Put price using the BS formula: $ 10.68
Put price using the put call parity formula: $ 10.68


# Comparing Black-Scholes formula to Monte-Carlo simulation


## Call options


In [9]:
bs_call_price = black_scholes_option_price(100, 110, 0.05, 1, 0.2, "call")
mc_call_price, mc_call_price_std = monte_carlo_option_price(
    100, 110, 0.05, 1, 0.2, 1000, "call"
)

print(f"Black-Scholes call price: $ {bs_call_price:.2f}")
print(f"Monte-Carlo call price: $ {mc_call_price:.2f} +/- {mc_call_price_std:.2f}")


Black-Scholes call price: $ 6.04
Monte-Carlo call price: $ 6.23 +/- 0.37


## Put options


In [11]:
bs_put_price = black_scholes_option_price(100, 110, 0.05, 1, 0.2, "put")
mc_put_price, mc_put_price_std = monte_carlo_option_price(
    100, 110, 0.05, 1, 0.2, 100000, "put"
)

print(f"Black-Scholes put price: $ {bs_put_price:.2f}")
print(f"Monte-Carlo put price: $ {mc_put_price:.2f} +/- {mc_put_price_std:.2f}")


Black-Scholes put price: $ 10.68
Monte-Carlo put price: $ 6.00 +/- 0.04
